# Dataset

In [39]:
import math

import torch
import pandas as pd
import numpy as np

In [40]:
from pathlib import Path

DATA_PATH = Path('data/')
DATA_PATH.mkdir(parents=True, exist_ok=True)

DATA_CACHE = Path('data/cache_dir/')
DATA_CACHE.mkdir(parents=True, exist_ok=True)

DATA_PATH_SAVE_MODELS = Path('data/models/')
DATA_PATH_SAVE_MODELS.mkdir(parents=True, exist_ok=True)

DATA_SYNTHETIC = Path('synthetic/')
DATA_SYNTHETIC.mkdir(parents=True, exist_ok=True)

pd.set_option('display.max_colwidth', 500) 

## [nanyy1025/covid_fake_news](https://huggingface.co/datasets/nanyy1025/covid_fake_news)

Датасет `nanyy1025/covid_fake_news` на Hugging Face содержит 10 700 записей, каждая из которых представляет собой твит, связанный с COVID-19, с меткой "real" (реальный) или "fake" (фейковый). Датасет разделен на три части: обучающая выборка (6 420 записей), валидационная выборка (2 140 записей) и тестовая выборка (2 140 записей). Данные представлены в формате CSV и предназначены для задач классификации текста и zero-shot классификации. Датасет был использован в исследовании "Fighting an Infodemic: COVID-19 Fake News Dataset" (arXiv:2011.03327). ([Hugging Face](https://huggingface.co/datasets/nanyy1025/covid_fake_news?utm_source=chatgpt.com)) 

```bibtex
@misc{patwa2020fighting,
title={Fighting an Infodemic: COVID-19 Fake News Dataset}, 
author={Parth Patwa and Shivam Sharma and Srinivas PYKL and Vineeth Guptha and Gitanjali Kumari and Md Shad Akhtar and Asif Ekbal and Amitava Das and Tanmoy Chakraborty},
year={2020},
eprint={2011.03327},
archivePrefix={arXiv},
primaryClass={cs.CL}
}
```

In [41]:
from datasets import load_dataset

In [42]:
def load_and_merge_nanyy1025_dataset(dataset_name: str = "nanyy1025/covid_fake_news",
                                     cache_dir=None) -> pd.DataFrame:
    dataset = load_dataset(dataset_name, cache_dir=cache_dir)

    train_df = dataset['train'].to_pandas()
    test_df = dataset['test'].to_pandas()
    validation_df = dataset['validation'].to_pandas()
    merged_df: pd.DataFrame = pd.concat([train_df, test_df, validation_df], ignore_index=True)
    merged_df.rename(columns={'tweet': 'text'}, inplace=True)
    return merged_df

nanyy1025_df = load_and_merge_nanyy1025_dataset( "nanyy1025/covid_fake_news", DATA_CACHE)

In [ ]:
nanyy1025_df.head()

In [ ]:
nanyy1025_df.info()

## [justinqbui/covid_fact_checked_google_api](https://huggingface.co/datasets/justinqbui/covid_fact_checked_google_api)

Датасет **justinqbui/covid_fact_checked_google_api** с Hugging Face представляет собой выборку проверенных фактов, связанных с COVID-19, собранных с использованием Google Fact Checker API. Вот его основные характеристики:

1. **Общий объем данных:**
   - Содержит 3 043 записи.
   - Первоначально было собрано 10 000 фактов, но для упрощения включены только те записи, где рейтинг был представлен одним словом — "false" (ложь) или "true" (правда). Около 90% фактов в датасете оценены как ложные.

2. **Модальности и форматы:**
   - Тип данных: текст.
   - Формат: CSV.
   - Также доступен в формате Parquet.

3. **Описание данных:**
   - Поля датасета:
     - `text`: текст проверенного факта.
     - `label`: метка правдивости (`true` или `false`).
   - Аннотации созданы экспертами.
   - Язык: английский (en-US).
   - Датасет является монолингвальным.

In [ ]:
def load_and_merge_justinqbui_dataset(dataset_name: str = "justinqbui/covid_fact_checked_google_api",
                                     cache_dir=None) -> pd.DataFrame:
    dataset = load_dataset(dataset_name, cache_dir=cache_dir)
    return dataset['train'].to_pandas()

justinqbui_1_df = load_and_merge_justinqbui_dataset( "justinqbui/covid_fact_checked_google_api", DATA_CACHE)

In [ ]:
justinqbui_1_df.head()

In [ ]:
justinqbui_1_df.info()

In [ ]:
justinqbui_1_df['label'].value_counts()

## [justinqbui/covid_fact_checked_polifact](https://huggingface.co/datasets/justinqbui/covid_fact_checked_polifact)

Датасет **justinqbui/covid_fact_checked_polifact** с Hugging Face включает записи, связанные с проверкой фактов о COVID-19, собранных с помощью автоматического веб-скрейпера, который извлек данные из PolitiFact COVID Fact Checker. Вот его ключевые характеристики:

1. **Общий объем данных:**
   - Содержит 1 190 записей.
   - Датасет состоит из утверждений и их оценки на правдивость.

2. **Модальности и форматы:**
   - Тип данных: текст.
   - Формат: CSV.
   - Также доступен в формате Parquet.

3. **Описание данных:**
   - Поля датасета:
     - `claim`: текст утверждения.
     - `rating`: оценка, присвоенная PolitiFact (7 значений: *half-true*, *full-flop*, *pants-fire*, *barely-true*, *true*, *mostly-true*, *false*).
     - `adjusted_rating`: упрощенная версия оценки (3 значения: например, "true", "false" и промежуточные варианты).
   - Поля предоставляют как детализированные, так и агрегированные оценки фактов.

In [ ]:
justinqbui_2_df = load_and_merge_justinqbui_dataset( "justinqbui/covid_fact_checked_polifact", DATA_CACHE)

In [ ]:
justinqbui_2_df.head()

In [ ]:
justinqbui_2_df.info()

In [ ]:
justinqbui_2_df['rating'].hist()

In [ ]:
justinqbui_2_df['rating'].value_counts()

In [ ]:
justinqbui_2_df['adjusted rating'].value_counts()

## Объединение

In [ ]:
nanyy1025_df['source'] = 'nanyy1025'
nanyy1025_df['original_label_1'] = nanyy1025_df['label']
nanyy1025_df['label_idx'] = nanyy1025_df['label'].apply(lambda x: 1 if x == 'fake' else 0)
nanyy1025_df = nanyy1025_df[['text', 'label_idx', 'original_label_1', 'source']]
nanyy1025_df.head(1)

In [ ]:
justinqbui_1_df['source'] = 'justinqbui_1'
justinqbui_1_df['original_label_1'] = justinqbui_1_df['label']
justinqbui_1_df['label_idx'] = justinqbui_1_df['label'].apply(lambda x: True if x == False else 0).astype(int)
justinqbui_1_df = justinqbui_1_df[['text', 'label_idx', 'original_label_1', 'source']]
justinqbui_1_df.head(1)

In [ ]:
justinqbui_2_df['source'] = 'justinqbui_2'
justinqbui_2_df['original_label_1'] = justinqbui_2_df['rating']
justinqbui_2_df['original_label_2'] = justinqbui_2_df['adjusted rating']

justinqbui_2_df['label_idx'] = justinqbui_2_df['adjusted rating'].apply(lambda x: 1 if x == 'false' else 0)

justinqbui_2_df = justinqbui_2_df[['claim', 'label_idx', 'original_label_1', 'original_label_2', 'source']]
justinqbui_2_df.rename(columns={'claim': 'text'}, inplace=True)
justinqbui_2_df.head(1)

In [58]:
combined_df = pd.concat([nanyy1025_df, justinqbui_1_df, justinqbui_2_df], ignore_index=True)

In [ ]:
combined_df.sample(5)

In [ ]:
combined_df.info()

In [ ]:
combined_df['original_label_1'].value_counts()

In [ ]:
combined_df['label_idx'].value_counts()

## Синтетика данных

In [63]:
negative_file = DATA_SYNTHETIC / 'negative_text.txt'
positive_file = DATA_SYNTHETIC / 'positive_text.txt'

def create_dataframe_from_file(file_path: Path, label_idx: int) -> pd.DataFrame:
    data = []
    with file_path.open('r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                data.append({'text': line, 'label_idx': label_idx})
    
    return pd.DataFrame(data)

negative_df = create_dataframe_from_file(negative_file, label_idx=1)
positive_df = create_dataframe_from_file(positive_file, label_idx=0)

synthetic_df = pd.concat([negative_df, positive_df], ignore_index=True)

In [ ]:
synthetic_df.info()

In [ ]:
synthetic_df.sample(5)

## Комментарии

In [66]:
from datasets import load_dataset
import pandas as pd

def load_reddit_dataset(dataset_name="beenakurian/reddit_comments_subreddit_canada", cache_dir=None) -> pd.DataFrame:
    dataset = load_dataset(dataset_name, cache_dir=cache_dir)
    df = dataset['train'].to_pandas()
    df.rename(columns={'comment': 'text'}, inplace=True)    
    df['original_label_1'] = df['sentiment']
    df['label_idx'] = 2  
    df['source'] = 'reddit_canada'
    return df[['text', 'label_idx', 'original_label_1', 'source']]

def load_toxic_comments_dataset(dataset_name="AiresPucrs/toxic-comments", cache_dir=None) -> pd.DataFrame:
    dataset = load_dataset(dataset_name, cache_dir=cache_dir)
    df = dataset['train'].to_pandas()
    df.rename(columns={'comment_text': 'text'}, inplace=True)
    df['original_label_1'] = df['toxic']
    df['label_idx'] = 2
    df['source'] = 'toxic_comments'
    return df[['text', 'label_idx', 'original_label_1', 'source']]

def load_twitter_dataset(dataset_name="gxb912/large-twitter-tweets-sentiment", cache_dir=None) -> pd.DataFrame:
    dataset = load_dataset(dataset_name, cache_dir=cache_dir)
    df_train = dataset['train'].to_pandas()
    df_test = dataset['test'].to_pandas()
    df = pd.concat([df_train, df_test], ignore_index=True)  # Исправлено объединение
    df['original_label_1'] = df['sentiment']
    df['label_idx'] = 2  
    df['source'] = 'twitter'  # Исправлено название источника
    return df[['text', 'label_idx', 'original_label_1', 'source']]


## [https://huggingface.co/datasets/gxb912/large-twitter-tweets-sentiment?row=62](https://huggingface.co/datasets/gxb912/large-twitter-tweets-sentiment?row=62)

In [67]:
reddit_df = load_reddit_dataset(cache_dir=DATA_CACHE)
toxic_df = load_toxic_comments_dataset(cache_dir=DATA_CACHE)
twitter_df = load_twitter_dataset(cache_dir=DATA_CACHE)

In [68]:
# toxic_df_sample = toxic_df.sample(10000)

In [69]:
# toxic_df_sample.sample(5)

## Данные размеченные руками

In [ ]:
import pandas as pd
from pathlib import Path

def load_and_process_data(file_path: Path) -> pd.DataFrame:
    """
    Загружает данные из Excel, добавляет дополнительные столбцы и возвращает обработанный DataFrame.
    Преобразует label_idx в int, а если возникают ошибки - ставит значение 2.

    :param file_path: Путь к файлу Excel.
    :return: Обработанный DataFrame.
    """
    data_marked_df = pd.read_excel(file_path).reset_index(drop=True)
    
    # Копируем new_label в label_idx
    data_marked_df['label_idx'] = data_marked_df['new_label']
    
    # Пробуем привести к int, если ошибка (например, NaN или строка), ставим 2
    data_marked_df['label_idx'] = pd.to_numeric(data_marked_df['label_idx'], errors='coerce').fillna(2).astype(int)
    
    # Добавляем источник
    data_marked_df['source'] = 'handle_marked'

    return data_marked_df[['text', 'label_idx', 'source']]

In [ ]:
data_marked_df = load_and_process_data(DATA_PATH / 'data_marked_df.xlsx')

# Общий датафрейм

In [32]:
synthetic_df['source'] = 'synthetic'
final_df = pd.concat([combined_df, synthetic_df, reddit_df, toxic_df, twitter_df, data_marked_df], ignore_index=True)
final_df["text"] = final_df["text"].replace("", None).drop_duplicates().fillna("")
final_df.to_excel(DATA_PATH / 'covid_vaccine_fake.xlsx', index=False)

In [33]:
idx2label = {
    0: "Real",
    1: "Fake",
    2: "Comments"
}

final_df['label_str'] = final_df['label_idx'].map(idx2label)

In [ ]:
final_df['label_str'].value_counts()

In [ ]:
final_df.sample(10)

In [36]:
import re

def clean_text(text):
    # text = re.sub(r'#\S+', '', text)  # Удаление хэштегов
    text = re.sub(r'@\S+', '', text)  # Удаление упоминаний
    text = re.sub(r'http\S+|www\.\S+', '', text)  # Удаление ссылок
    return text.strip()  # Удаление лишних пробелов

final_df['text'] = final_df['text'].apply(clean_text)

In [ ]:
final_df.sample(10)

In [38]:
final_df.to_excel(DATA_PATH / 'covid_vaccine_fake_clear.xlsx', index=False)